Tha backbone for this notebook is collected from: 

https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/data-api-tutorials/search_and_download_quickstart.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Reading, processing and storing of JSON files

In [ ]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth

# API Key stored as an env variable
PLANET_API_KEY = 'your API key'

# Define the file path to the folder containing the GeoJSON files
folder_path = 'Your storage path'

# Loop through the GeoJSON files in the folder and load their content as dictionaries
geojson_data = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as f:
            data = json.load(f)
            #get the type
            Type = data['geometryType']
            my_list = list(data.values())
            geom = my_list[5][0]['geometry']['rings'] #extracting geometry values
            new_data = {'type': Type, 'coordinates': geom}
            new_data['type'] = new_data['type'].replace('esriGeometryPolygon', 'Polygon') #changing type for planet API
            geojson_data.append(new_data) 


In [ ]:
#Look first element of the geometry list
len(geojson_data)

# Creating Filters

In [ ]:
# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-08-31T00:00:00.000Z",
    "lte": "2016-09-01T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

In [ ]:
geom_filter_list =[] #we are making a list as there are more than one geometry filters
for geojson_geometry in geojson_data:
  geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geojson_geometry
  }
  geom_filter_list.append(geometry_filter)


In [ ]:
#look at the first element
geom_filter_list[0]

#Applying filters to prepare search result

In [ ]:
item_type = "PSScene"
PLANET_API_KEY = 'Your API key'
collected_geojson = []

for geometry_filter in geom_filter_list:
  combined_filter = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
  }
  search_request = {
      "item_types": [item_type], 
       "filter": combined_filter
        }
  search_result = \
      requests.post(
     'https://api.planet.com/data/v1/quick-search',
      auth=HTTPBasicAuth(PLANET_API_KEY, ''),
      json=search_request)
  geojson = search_result.json()
  collected_geojson.append(geojson)


The above steps take time everytime you run the code. Its better to store the geojson and reuse it for future run. That is shown in the following code. Use the pickling step and save your file. Once you save, for the later runs just load the pickles and comment out the pickling part.

In [ ]:
#save the collected jsons to a list for reuse

#pickling
import pickle
with open("test", "wb") as fp: 
  pickle.dump(collected_geojson, fp)
 
#open the pickle
with open("collected_geojson", "rb") as fp:   # Unpickling
  collected_geojson = pickle.load(fp) 

In [ ]:
#Look at the list of collected geojson which we will use later
collected_geojson

In [ ]:
#lets see the first Item
collected_geojson[0]

# Activation & Downloading

In [ ]:
# Parse out useful links
def activation_basic_analytic_4b():
  links = result.json()[u"basic_analytic_4b"]["_links"]
  self_link = links["_self"]
  activation_link = links["activate"]

  # Request activation of the 'basic_analytic_4b' asset:
  activate_result = \
    requests.get(
      activation_link,
      auth=HTTPBasicAuth(PLANET_API_KEY, '')
    )
  activation_status_result = \
    requests.get(
    self_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
    )
    
  return activation_status_result.json()["location"]


  # Parse out useful links
def activation_ortho_analytic_4b():
  links = result.json()[u"ortho_analytic_4b"]["_links"]
  self_link = links["_self"]
  activation_link = links["activate"]

  # Request activation of the 'basic_analytic_4b' asset:
  activate_result = \
    requests.get(
      activation_link,
      auth=HTTPBasicAuth(PLANET_API_KEY, '')
    )
  activation_status_result = \
    requests.get(
    self_link,
    auth=HTTPBasicAuth(PLANET_API_KEY, '')
    )
    
  return activation_status_result.json()["location"]

# Downloading different asset types

In [ ]:
import os
import json
import requests
from requests.auth import HTTPBasicAuth


# API Key stored as an env variable
PLANET_API_KEY = 'Your API key'
item_type = "PSScene"

headers = {"Authorization": PLANET_API_KEY}

In [ ]:
#for all asset [making a dictionary]

id_url_dict = {}
for geojson in collected_geojson[0:100]:
    image_ids = [feature['id'] for feature in geojson['features']]
    for image_id in image_ids:
        id_url_dict[image_id] = []
        id_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, image_id)
        result = requests.get(id_url, auth=HTTPBasicAuth(PLANET_API_KEY, ''))
        try:
          link_1 = activation_basic_analytic_4b()
          link_2 = activation_ortho_analytic_4b()
          id_url_dict[image_id] = [link_1, link_2]
        except :
          print("asset not available")
        

In [ ]:
id_url_dict

# Dump the links to an Excel file

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(id_url_dict, orient="index")
df.to_csv("dl_links.csv", header=["Download Link for Basic Analytic", "Download Link for Ortho Analytic"])
